In [7]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from IPython.display import Image, display
import pandas as pd
from typing import Any, Annotated
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
import numpy as np

In [3]:
import plotly.express as px
import json

def plotly_to_echarts(fig):
    echarts_json = {
        "title": {"text": fig.layout.title.text if fig.layout.title else ""},
        "tooltip": {"trigger": "axis"},
        "legend": {"data": []},
        "xAxis": {"type": "category", "data": []},
        "yAxis": {"type": "value"},
        "series": []
    }
    
    for trace in fig.data:
        series_data = {
            "name": trace.name if trace.name else "Series",
            "type": "line" if trace.mode == "lines" else "bar",
            "data": []
        }
        
        if "x" in trace:
            echarts_json["xAxis"]["data"] = trace.x.tolist()
        print("type trace.y",type(trace.y))
        if "y" in trace:
            series_data["data"] = trace.y.tolist()
        
        echarts_json["series"].append(series_data)
        echarts_json["legend"]["data"].append(series_data["name"])
    print("echarts_json::",type(echarts_json))
    return echarts_json

# Example usage:
fig = px.line(x=[1, 2, 3, 4], y=[10, 20, 15, 25], title="Example Plotly Chart")
echarts_json = plotly_to_echarts(fig)
print(json.dumps(echarts_json))

type trace.y <class 'numpy.ndarray'>
echarts_json:: <class 'dict'>
{"title": {"text": "Example Plotly Chart"}, "tooltip": {"trigger": "axis"}, "legend": {"data": ["Series"]}, "xAxis": {"type": "category", "data": [1, 2, 3, 4]}, "yAxis": {"type": "value"}, "series": [{"name": "Series", "type": "line", "data": [10, 20, 15, 25]}]}


In [27]:
for trace in fig.data:
    x_data = trace.x
    y_data = trace.y
    print(f"X Data: {x_data}")
    print(f"Y Data: {y_data}")

X Data: [1 2 3 4]
Y Data: [10 20 15 25]


In [29]:
fig.data

(Scatter({
     'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
     'legendgroup': '',
     'line': {'color': '#636efa', 'dash': 'solid'},
     'marker': {'symbol': 'circle'},
     'mode': 'lines',
     'name': '',
     'orientation': 'v',
     'showlegend': False,
     'x': array([1, 2, 3, 4]),
     'xaxis': 'x',
     'y': array([10, 20, 15, 25]),
     'yaxis': 'y'
 }),)

In [31]:
import plotly.express as px
df = px.data.gapminder().query("continent == 'Oceania'")
fig = px.line(df, x='year', y='lifeExp', color='country', markers=True)
fig.data

(Scatter({
     'hovertemplate': 'country=Australia<br>year=%{x}<br>lifeExp=%{y}<extra></extra>',
     'legendgroup': 'Australia',
     'line': {'color': '#636efa', 'dash': 'solid'},
     'marker': {'symbol': 'circle'},
     'mode': 'lines+markers',
     'name': 'Australia',
     'orientation': 'v',
     'showlegend': True,
     'x': array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002, 2007]),
     'xaxis': 'x',
     'y': array([69.12 , 70.33 , 70.93 , 71.1  , 71.93 , 73.49 , 74.74 , 76.32 , 77.56 ,
                 78.83 , 80.37 , 81.235]),
     'yaxis': 'y'
 }),
 Scatter({
     'hovertemplate': 'country=New Zealand<br>year=%{x}<br>lifeExp=%{y}<extra></extra>',
     'legendgroup': 'New Zealand',
     'line': {'color': '#EF553B', 'dash': 'solid'},
     'marker': {'symbol': 'circle'},
     'mode': 'lines+markers',
     'name': 'New Zealand',
     'orientation': 'v',
     'showlegend': True,
     'x': array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 

In [16]:
fig_data['layout']

{'template': {'data': {'histogram2dcontour': [{'type': 'histogram2dcontour',
     'colorbar': {'outlinewidth': 0, 'ticks': ''},
     'colorscale': [[0.0, '#0d0887'],
      [0.1111111111111111, '#46039f'],
      [0.2222222222222222, '#7201a8'],
      [0.3333333333333333, '#9c179e'],
      [0.4444444444444444, '#bd3786'],
      [0.5555555555555556, '#d8576b'],
      [0.6666666666666666, '#ed7953'],
      [0.7777777777777778, '#fb9f3a'],
      [0.8888888888888888, '#fdca26'],
      [1.0, '#f0f921']]}],
   'choropleth': [{'type': 'choropleth',
     'colorbar': {'outlinewidth': 0, 'ticks': ''}}],
   'histogram2d': [{'type': 'histogram2d',
     'colorbar': {'outlinewidth': 0, 'ticks': ''},
     'colorscale': [[0.0, '#0d0887'],
      [0.1111111111111111, '#46039f'],
      [0.2222222222222222, '#7201a8'],
      [0.3333333333333333, '#9c179e'],
      [0.4444444444444444, '#bd3786'],
      [0.5555555555555556, '#d8576b'],
      [0.6666666666666666, '#ed7953'],
      [0.7777777777777778, '#fb9f3a

In [5]:
GROQ_API_KEY = "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"

In [9]:
llm = ChatGroq(temperature=0.1, groq_api_key=GROQ_API_KEY, model_name="llama-3.3-70b-specdec")

In [ ]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        error : Binary flag for control flow to indicate whether test error was tripped
        messages : With user question, error messages, reasoning
        generation : Code solution
        iterations : Number of tries
    """    
    error: str
    data_path: str
    question: str
    input_data: dict[str, Any]
    data_schema: str
    data_summary: str
    messages: Annotated[list, add_messages]
    user_query: str
    code: str
    final_results: dict[str, Any]
    

    # class Config:
    #     arbitrary_types_allowed = True

In [ ]:
def json_df(json_str)->pd.DataFrame:
    import pandas as pd
    from io import StringIO
    json_buffer = StringIO(json_str)
    df = pd.read_json(json_buffer, orient='split')
    return df

In [ ]:
# inside execute.py

import io
import sys

def execute_generated_code(code, df):
    print("Started executing generated code")
    # Clean the code by removing block markers and spaces
    # clean_code = code.strip('```').strip()  
    clean_code = code.replace('```','###')
    clean_code = clean_code.replace("fig.show()","# fig.show()")
    print('111::',clean_code)
    # Redirect stdout to capture print statements
    # output_buffer = io.StringIO()
    # sys.stdout = output_buffer  
    print('222')
    # Pass your DataFrame to the local scope
    local_scope = {"df": df}
    print('333')
    try:
        print('444')
        # Execute the code with the DataFrame in the local scope
        exec(clean_code, {}, local_scope)
        print('555')
    except Exception as e:
        print("Error executing generated code::",e)
        return f"Error during execution: {e}"
    finally:
        print("finally")
        # Reset stdout to default after execution
        # sys.stdout = sys.__stdout__
    print("befre return")
    # Return both the output captured and the local scope (for plots or other variables)
    # output = output_buffer.getvalue()
    print("Ended executing generated code")
    return local_scope

In [ ]:
def load_data(state: GraphState):
    print(state["data_path"])
    df = pd.read_csv(state["data_path"])
    # df = pd.DataFrame({"name": ["Alice", "Bob"]})
    # print(df.shape)
    return {"input_data":df.to_json(orient='split')}

In [ ]:
def summarize_data(state: GraphState):
    
    df_json = state['input_data']
    df = None
    if df_json is not None:
        df = json_df(df_json)
        print("summarize_data df shape:", df.shape)
    else:
        return {"data_summary": None}
    columns = list(df.columns)
    data_types = df.dtypes.to_dict()

    unique_values = {}

    # for column in df.columns:
    #     if df[column].dtype == 'object' or df[column].dtype.name == 'category':
    #         unique_values[column] = df[column].unique().tolist()
    #     elif 'datetime' in str(df[column].dtype):
    #         date_sample = df[column].dropna().iloc[0]
    #         inferred_format = pd.to_datetime([date_sample]).strftime('%Y-%m-%d')[0]
    #         unique_values[column] = f"Date format: {inferred_format}"


    summary = (
        f"Columns: {columns}\n"
        f"Data Types: {data_types}\n"
        # f"Unique Values for Categorical/Date Columns:\n{unique_values}"
    )
    
    return {"data_summary":summary}

In [ ]:
def human_userinput1(state):
    print("---human_input1---")    
    user_msg = interrupt("Enter query")
    # user_msg = "What is the data schema?"
    # return {"user_query": user_msg, "messages": [HumanMessage(user_msg)]}
    return {"messages": [HumanMessage(user_msg)]}

In [ ]:
# def generate_code1(state: GraphState):
#     print("---generate_code---")
#     data_summary = state["data_summary"]
#     # query = state["user_query"]
#     message = state["messages"][-1].content
#     print("message:", message)
#     system_message = (
#         f"You are a professional data analyst proficient in Python programming.\n"
#         f"Based on the following dataset summary:\n{data_summary}\n"
#         "Provide clean python code using pandas and plotly (only the code, nothing else) that when executed will:\n"
#         "The code should contain a visualization that is well labelled."
#         "Assume the dataset has been read into a dataframe called df." 
#         "If the query involves generating a chart (e.g., bar plot), please ensure the chart displays no more than 10 categories (top or bottom)."
#         "If the user specifically requests to show all categories, include that in the code. If no specific instruction is provided, limit the chart to the top or bottom 10 categories based on the count or value"
#         f" Answer the user's question: '{message}'.\n"
#     )
#     print("before llm generate",system_message)
#     response = llm.invoke(system_message)
#     print("after llm generate",response)
#     return {"code": response.content}

In [ ]:
def generate_code(state: GraphState):
    print("---generate_code---")
    data_summary = state["data_summary"]
    # query = state["user_query"]
    message = state["messages"][-1].content
    print("message:", message)
    system_message = (
        f"You are a professional data analyst proficient in Python programming.\n"
        f"Based on the following dataset summary:\n{data_summary}\n"
        "Provide clean python code using pandas and plotly (only the code, nothing else) that when executed will:\n"
        "The code should contain a visualization that is well labelled.\n"
        "Assume the dataset has been read into a dataframe called df.\n" 
        "If the query involves generating a chart (e.g., bar plot), please ensure the chart displays no more than 10 categories (top or bottom).\n"
        "If the user specifically requests to show all categories, include that in the code. If no specific instruction is provided, limit the chart to the top or bottom 10 categories based on the count or value\n"
        "Make sure all the important results are saved as a dictionary with name 'final_results'. Each result item should be accompanied with description about that item.\n"
        "For plotly fig object, include type as 'figure' and None for remaining in final results"
        f" Answer the user's question: '{message}'.\n"
    )
    print("before llm generate",system_message)
    response = llm.invoke(system_message)
    print("after llm generate",response)
    return {"code": response.content}

In [ ]:
def convert_arrays_to_lists(obj):
    if isinstance(obj, np.ndarray) or isinstance(obj, pd.Series):
        return obj.tolist()  # Convert NumPy arrays and Pandas Series to lists
    elif isinstance(obj, dict):
        return {key: convert_arrays_to_lists(value) for key, value in obj.items()}  # Recursively process dict
    elif isinstance(obj, list):
        return [convert_arrays_to_lists(item) for item in obj]  # Recursively process lists
    else:
        return obj  # Return as is if it's not an array/series

In [ ]:
def execute_code(state: GraphState):
    print("-----Executing code----")
    gen_code = state["code"]
    data = json_df(state["input_data"])
    results = execute_generated_code(gen_code, data)
    if "df" in results:
        del results["df"]
    
    if results is not None and 'final_results' in results:
        final_results = convert_arrays_to_lists(results['final_results'])
        for k,v in final_results.items():
            if "type" in v and str.lower(v["type"]) == "figure":
                final_results[k]['value']=plotly_to_echarts(v['value'])
        print("final_results::",final_results)
        return {"final_results":final_results}
    else:
        return {"final_results":None}

In [ ]:
def prepare_answer(state: GraphState):
    print("----prepare answer-----")
    results = state["final_results"]
    print("Results",results)
    return None
    
        

In [ ]:
builder = StateGraph(GraphState)

builder.add_node("load_data", load_data)
builder.add_node("summarize_data", summarize_data)
builder.add_node("human_userinput1", human_userinput1)
builder.add_node("generate_code", generate_code)
builder.add_node("execute_code", execute_code)
builder.add_node("prepare_answer", prepare_answer)
builder.add_edge(START, "load_data")
builder.add_edge("load_data", "summarize_data")
builder.add_edge("summarize_data", "human_userinput1")
builder.add_edge("human_userinput1", "generate_code")
builder.add_edge("generate_code", "execute_code")
builder.add_edge("execute_code", "prepare_answer")
builder.add_edge("prepare_answer", END)


In [ ]:
# Set up memory
memory = MemorySaver()
# memory = SqliteSaver.from_conn_string(":memory:")
# Add
graph = builder.compile(checkpointer=memory)
# graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Input
initial_input = {"data_path": r"D:\suresh\work\projects\W360_MVP1\playground\notebooks\data\Electric_Vehicle_Population_Data.csv"}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    # print(event)
    print("\n")

In [ ]:
graph.get_state(thread).next

In [ ]:
user_text = "How many counties are there?"
for event in graph.stream(Command(resume=user_text), thread, stream_mode="values"):
        # print(event)
        print("\n")


In [ ]:
for k, v in event.items():
    print(k)

In [ ]:
graph.get_state(thread).next

In [ ]:
x = {"a":10,"b":20}
print(x)

In [ ]:
str(x)

In [ ]:
df_json = graph.get_state(thread).values['input_data']
input_df = json_df(df_json)
print(input_df.shape)

In [ ]:
!pip show plotly

In [ ]:
code1 = graph.get_state(thread).values['code'].replace("```","")
print(code1)

In [ ]:
input_df.head()

In [ ]:
code = """```python
import pandas as pd
import plotly.express as px

# Get the number of unique counties
num_counties = df['County'].nunique()
print(f"There are {num_counties} counties.")

# Get the top 10 counties by count
top_10_counties = df['County'].value_counts().head(10)

# Create a bar plot
fig = px.bar(x=top_10_counties.index, y=top_10_counties.values, title="Top 10 Counties by Count", labels={'x': 'County', 'y': 'Count'})
fig.show()
```"""

In [ ]:
print(code)

In [ ]:
clean_code = code.strip('```').strip()  
print(clean_code)

In [ ]:
import io
import sys

In [ ]:
output_buffer = io.StringIO()
sys.stdout = output_buffer 

In [ ]:
clean_code = code.strip('```').strip()  
print('111::')
# Redirect stdout to capture print statements
output_buffer = io.StringIO()
sys.stdout = output_buffer  
print('222')
# Pass your DataFrame to the local scope
local_scope = {"df": df}
print('333')


In [ ]:
df = pd.read_csv(r"D:\suresh\work\projects\W360_MVP1\playground\notebooks\data\Electric_Vehicle_Population_Data.csv")

In [ ]:
###python
import pandas as pd
import plotly.express as px

# Get the number of unique counties
num_counties = df['County'].nunique()
print(f"There are {num_counties} counties.")

# Get the top 10 counties by count
top_10_counties = df['County'].value_counts().head(10)

# Create a bar plot
fig = px.bar(x=top_10_counties.index, y=top_10_counties.values, title="Top 10 Counties by Count", labels={"x": "County", "y": "Count"})
# fig.show()
###